In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import os

In [2]:
# Define image size and directories
IMAGE_SIZE = [299, 299]
train_directory = r"C:\Users\ken\OneDrive\Desktop\img.class\melon_dataset\train"
test_directory = r"C:\Users\ken\OneDrive\Desktop\img.class\melon_dataset\test"
val_directory = r"C:\Users\ken\OneDrive\Desktop\img.class\melon_dataset\val"

In [3]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [4]:
# Data augmentation for testing set (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# Load training and testing sets with data augmentation
training_set = train_datagen.flow_from_directory(
    train_directory,
    target_size=(299, 299),  # Adjust based on InceptionV3 input size
    batch_size=32,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_directory,
    target_size=(299, 299),  # Adjust based on InceptionV3 input size
    batch_size=32,
    class_mode='categorical'
)

Found 1615 images belonging to 10 classes.
Found 550 images belonging to 10 classes.


In [6]:
# Add preprocessing layer to the front of InceptionV3
inception = InceptionV3(input_shape=(299, 299, 3), weights='imagenet', include_top=False)



87910968/87910968 [==============================] - 5s 0us/step


In [7]:
# Freeze the layers
for layer in inception.layers:
    layer.trainable = False
    
    # Global Average Pooling
x = GlobalAveragePooling2D()(inception.output)

# Fully Connected Layer
x = Dense(128, activation='relu')(x)

# Output Layer
output_tensor = Dense(10, activation='softmax')(x)

# Create the model
model = Model(inputs=inception.input, outputs=output_tensor)

In [8]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [9]:
# Train the model
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

Epoch 1/10


51/51 [==============================] - 122s 2s/step - loss: 0.9603 - accuracy: 0.7040 - val_loss: 0.4224 - val_accuracy: 0.8673
Epoch 2/10
51/51 [==============================] - 112s 2s/step - loss: 0.3462 - accuracy: 0.8910 - val_loss: 0.2753 - val_accuracy: 0.9055
Epoch 3/10
51/51 [==============================] - 103s 2s/step - loss: 0.2203 - accuracy: 0.9356 - val_loss: 0.2876 - val_accuracy: 0.9036
Epoch 4/10
51/51 [==============================] - 105s 2s/step - loss: 0.2073 - accuracy: 0.9399 - val_loss: 0.2185 - val_accuracy: 0.9255
Epoch 5/10
51/51 [==============================] - 111s 2s/step - loss: 0.1341 - accuracy: 0.9659 - val_loss: 0.1803 - val_accuracy: 0.9436
Epoch 6/10
51/51 [==============================] - 124s 2s/step - loss: 0.1063 - accuracy: 0.9728 - val_loss: 0.1431 - val_accuracy: 0.9600
Epoch 7/10
51/51 [==============================] - 116s 2s/step - loss: 0.0859 - accuracy: 0.9789 - val_loss: 0.1517 - val_accuracy: 0.9491
Epoch 8/10


In [10]:
# Save the model
model.save('melon_species_Inceptionv3.h5')

C:\Users\ken\anaconda31\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
# Plot loss and accuracy curves
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.legend()